In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
import thor.data as data
import thor.utils as gutils
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag

notebook_name = "cpol_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
# Create the data_options dictionary
converted_options = {"save": False, "load": True, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    converted_options=converted_options,
    parent_local="/home/ewan/Documents/THOR/thor/test/data/converted")
era5_pl_options = data.era5.data_options(parent_local="/media/ewan/ewan_drive/era5")
era5_sl_options = data.era5.data_options(data_format="single-levels", parent_local="/media/ewan/ewan_drive/era5")
data_options = option.consolidate_options([cpol_options, era5_pl_options, era5_sl_options])
dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="cpol_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
filepath = data_options["cpol"]["filepaths"][0]
cpol = xr.open_dataset(filepath)
# lats = cpol["point_latitude"].isel(z=0).values
# lons = cpol["point_longitude"].isel(z=0).values
# latitude, longitude = grid.new_geographic_grid(lats, lons, .025, .025)
latitude, longitude = [cpol.latitude.values, cpol.longitude.values]
grid_options = grid.create_options(name="geographic", latitude=latitude, longitude=longitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol", tags=["era5_pl", "era5_sl"])
option.save_track_options(track_options, filename="cpol_mcs")


2024-05-18 17:34:39,436 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/data_options/cpol_era5.yaml
2024-05-18 17:34:39,515 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/grid_options/cpol_geographic.yaml
2024-05-18 17:34:39,527 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/tag_options/era5.yaml
2024-05-18 17:34:39,529 - thor.option - DEBUG - Saving options to /home/ewan/Documents/THOR/thor/options/track_options/cpol_mcs.yaml


In [7]:
times = data.aura.generate_cpol_times(data_options["cpol"]["filepaths"])
tracks = track.simultaneous_track(times, data_options, grid_options, track_options, tag_options)

2024-05-17 21:49:51,824 - thor.track - INFO - Processing 2005-11-13T00:00:09.000000000.
2024-05-17 21:49:51,825 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T00:00:09.000000000
2024-05-17 21:49:51,857 - thor.track - DEBUG - Processing hierarchy level 0.
2024-05-17 21:49:51,858 - thor.track - DEBUG - Tracking cell.
2024-05-17 21:49:51,860 - thor.track - DEBUG - Tracking middle_cloud.
2024-05-17 21:49:51,860 - thor.track - DEBUG - Tracking anvil.
2024-05-17 21:49:51,861 - thor.track - DEBUG - Processing hierarchy level 1.
2024-05-17 21:49:51,863 - thor.track - DEBUG - Tracking mcs.
2024-05-17 21:49:51,864 - thor.data.era5 - DEBUG - Updating era5_pl dataset for 2005-11-13T00:00:09.000000000
2024-05-17 21:49:51,866 - thor.data.era5 - DEBUG - Subsetting u_era5_oper_pl_20051101-20051130.nc
2024-05-17 21:49:52,042 - thor.data.era5 - DEBUG - Subsetting v_era5_oper_pl_20051101-20051130.nc
2024-05-17 21:49:52,201 - thor.data.era5 - DEBUG - Subsetting z_era5_oper_pl_20051101-200

In [5]:
track_options[0]["anvil"]

{'name': 'anvil',
 'hierarchy_level': 0,
 'deque_length': 2,
 'mask_options': {'save': False, 'load': False},
 'tags': None,
 'dataset': 'cpol',
 'variable': 'reflectivity',
 'time_interp': None,
 'detection': {'method': 'threshold',
  'altitudes': [7500, 10000],
  'flatten': 'max',
  'threshold': 15},
 'tracking': {'method': 'tint',
  'options': {'search_margin': 4000,
   'flow_margin': 40000,
   'max_disparity': 999,
   'max_flow_mag': 50,
   'max_shift_disparity': 15,
   'global_shift_altitude': 8000}}}

In [ ]:
ds = xr.open_mfdataset(f"/media/ewan/ewan_drive/era5/pressure-levels/reanalysis/u/2005/u_era5_oper_pl_*.nc", engine="netcdf4", parallel=True)
ds = ds.chunk({"time": 12, "level": 37, "latitude": 300, "longitude": 300})


In [ ]:
ds

<xarray.Dataset> Size: 450GB
Dimensions:    (longitude: 1440, latitude: 721, level: 37, time: 1464)
Coordinates:
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 148B 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 12kB 2005-11-01 ... 2005-12-31T23:00:00
Data variables:
    u          (time, level, latitude, longitude) float64 450GB dask.array<chunksize=(12, 37, 400, 400), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-08 05:52:10 UTC+1100 by era5_replication_tools-1.2....
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 pressure-levels oper u_component_of_wind 20051101-2005...

In [ ]:

ds = ds.chunk({"time": 12, "level": 37, "latitude": 400, "longitude": 400})
ds = ds.isel(time=slice(11,14), latitude=slice(300, 350), longitude=slice(300, 350))
ds.load()
# ds.nbytes/1e6
ds


In [ ]:

times = np.arange(np.datetime64("2005-11-05T15:40:00"), np.datetime64("2005-11-05T19:30:00"), np.timedelta64(10, "m")).astype("datetime64[ns]")
ds = ds.sel(time=times[0], latitude=slice(0, 10), longitude=slice(0, 10))
ds

In [96]:
gutils.get_hour_interval(np.datetime64("2022-01-03T04:55"), interval=12)

(numpy.datetime64('2022-01-03T00','h'), numpy.datetime64('2022-01-03T12','h'))

In [92]:
ds = xr.open_dataset(f"/media/ewan/ewan_drive/era5/pressure-levels/reanalysis/u/2005/u_era5_oper_pl_20051101-20051131.nc")
ds

<xarray.Dataset> Size: 221GB
Dimensions:    (longitude: 1440, latitude: 721, level: 37, time: 720)
Coordinates:
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 148B 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 6kB 2005-11-01 ... 2005-11-30T23:00:00
Data variables:
    u          (time, level, latitude, longitude) float64 221GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-08 05:52:10 UTC+1100 by era5_replication_tools-1.2....
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 pressure-levels oper u_component_of_wind 20051101-2005...

In [ ]:
ds = xr.open_dataset(f"/media/ewan/ewan_drive/era5/pressure-levels/reanalysis/{var}/2005/{var}_era5_oper_pl_20051101-20051131.nc")

In [ ]:
ds_list = []
for var in ["u", "v", "z", "r", "t"]:
    ds = xr.open_dataset(f"/media/ewan/ewan_drive/era5/pressure-levels/reanalysis/{var}/2005/{var}_era5_oper_pl_20051101-20051131.nc")
    ds = ds.chunk({"time": 2, "level": 37, "latitude": 50, "longitude": 50})
    times = np.arange(np.datetime64("2005-11-05T15:40:00"), np.datetime64("2005-11-05T19:30:00"), np.timedelta64(10, "m")).astype("datetime64[ns]")
    # with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        # ds = ds.sel(latitude=slice(33, 29), longitude=slice(-13, -9))
    ds = ds.interp(time=times[:4], latitude=np.linspace(30, 32, 5), longitude=np.linspace(-12, -10, 5), method="linear").compute()
    ds_list.append(ds)
xr.merge(ds_list)
ds

<xarray.Dataset> Size: 30kB
Dimensions:    (level: 37, time: 4, latitude: 5, longitude: 5)
Coordinates:
  * level      (level) int32 148B 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 32B 2005-11-05T15:40:00 ... 2005-11-05T1...
  * latitude   (latitude) float64 40B 30.0 30.5 31.0 31.5 32.0
  * longitude  (longitude) float64 40B -12.0 -11.5 -11.0 -10.5 -10.0
Data variables:
    t          (time, level, latitude, longitude) float64 30kB 259.9 ... 290.5
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-08 04:59:56 UTC+1100 by era5_replication_tools-1.2....
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...
    title:        ERA5 pressure-levels oper temperature 20051101-20051130